In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pickle
from sklearn.metrics import mean_absolute_error
import warnings
import xgboost as xgb
warnings.filterwarnings("ignore")


In [2]:
df=pd.read_csv('FINAL.csv')

In [3]:
df.columns

Index(['match_id', 'innings', 'Unnamed: 2', 'season', 'start_date', 'venue',
       'ball', 'batting_team', 'bowling_team', 'striker', 'non_striker',
       'bowler', 'runs_off_bat', 'extras', 'wides', 'noballs', 'byes',
       'legbyes', 'penalty', 'wicket_type', 'player_dismissed',
       'other_wicket_type', 'other_player_dismissed', 'bowler_type',
       'Total_runs', 'Extraballs', 'over_type', 'number_of_wicket',
       'number_of_dotballs', 'Batting_type'],
      dtype='object')

In [4]:
df['batting_team'].unique()

array(['Kolkata Knight Riders', 'Royal Challengers Bangalore',
       'Chennai Super Kings', 'Kings XI Punjab', 'Rajasthan Royals',
       'Delhi Daredevils', 'Mumbai Indians', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Rising Pune Supergiants', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Delhi Capitals', 'Punjab Kings'],
      dtype=object)

In [5]:
for index,row in enumerate(df['batting_team']):
    if row=='Delhi Daredevils':
        df.loc[index,'batting_team']='Delhi Capitals'
    if row=='Deccan Chargers':
        df.loc[index,'batting_team']='Sunrisers Hyderabad'
    if row=='Kings XI Punjab':
        df.loc[index,'batting_team']='Punjab Kings'

In [6]:
for index,row in enumerate(df['bowling_team']):
    if row=='Delhi Daredevils':
        df.loc[index,'bowling_team']='Delhi Capitals'
    if row=='Deccan Chargers':
        df.loc[index,'bowling_team']='Sunrisers Hyderabad'
    if row=='Kings XI Punjab':
        df.loc[index,'bowling_team']='Punjab Kings'

In [7]:
df['batting_team'].unique()

array(['Kolkata Knight Riders', 'Royal Challengers Bangalore',
       'Chennai Super Kings', 'Punjab Kings', 'Rajasthan Royals',
       'Delhi Capitals', 'Mumbai Indians', 'Sunrisers Hyderabad',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Rising Pune Supergiants',
       'Gujarat Lions', 'Rising Pune Supergiant'], dtype=object)

In [8]:
consistent_team=['Kolkata Knight Riders', 'Royal Challengers Bangalore','Chennai Super Kings',  'Rajasthan Royals','Delhi Capitals', 'Mumbai Indians', 'Sunrisers Hyderabad', 'Punjab Kings']

In [9]:
df['batting_team'].isin(consistent_team)

0         True
1         True
2         True
3         True
4         True
          ... 
200658    True
200659    True
200660    True
200661    True
200662    True
Name: batting_team, Length: 200663, dtype: bool

In [10]:
df=df[df['batting_team'].isin(consistent_team)]

In [11]:
df=df[df['bowling_team'].isin(consistent_team)]

In [12]:
df

,match_id,innings,Unnamed: 2,season,start_date,venue,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,bowler_type,Total_runs,Extraballs,over_type,number_of_wicket,number_of_dotballs,Batting_type
0,335982,1,0,2008,2008-04-18,Chinnaswamy Stadium(Banglore),0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,0,1,NaN,NaN,NaN,1.0,NaN,NaN,0,NaN,NaN,Right_arm_medium,1,0.0,PowerPlay,1.0,0.0,Left_hand_batsman
1,335982,1,112,2008,2008-04-18,Chinnaswamy Stadium(Banglore),0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,Right_arm_medium,0,0.0,PowerPlay,1.0,1.0,Right_hand_batsman
2,335982,1,143,2008,2008-04-18,Chinnaswamy Stadium(Banglore),0.3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,1,1.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,Right_arm_medium,1,0.0,PowerPlay,1.0,0.0,Right_hand_batsman
3,335982,1,144,2008,2008-04-18,Chinnaswamy Stadium(Banglore),0.4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,Right_arm_medium,0,0.0,PowerPlay,1.0,1.0,Right_hand_batsman
4,335982,1,145,2008,2008-04-18,Chinnaswamy Stadium(Banglore),0.5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,Right_arm_medium,0,0.0,PowerPlay,1.0,1.0,Right_hand_batsman
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200658,1254086,2,200331,2021,2021-05-02,Motera Stadium(Ahemdabad),17.2,Delhi Capitals,Punjab Kings,SO Hetmyer,S Dhawan,RP Meredith,6,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,Right_arm_fast,6,0.0,Death overs,1.0,0.0,Left_hand_batsman
200659,1254086,2,200330,2021,2021-05-02,Motera Stadium(Ahemdabad),17.3,Delhi Capitals,Punjab Kings,SO Hetmyer,S Dhawan,RP Meredith,6,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,Right_arm_fast,6,0.0,Death overs,1.0,0.0,Left_hand_batsman
200660,1254086,2,200329,2021,2021-05-02,Motera Stadium(Ahemdabad),17.4,Delhi Capitals,Punjab Kings,SO Hetmyer,S Dhawan,RP Meredith,4,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,Right_arm_fast,4,0.0,Death overs,1.0,0.0,Left_hand_batsman
200661,1254086,2,200328,2021,2021-05-02,Motera Stadium(Ahemdabad),17.5,Delhi Capitals,Punjab Kings,SO Hetmyer,S Dhawan,RP Meredith,0,1,1.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,Right_arm_fast,1,0.0,Death overs,1.0,0.0,Left_hand_batsman


In [13]:
df['ball']

0          0.1
1          0.2
2          0.3
3          0.4
4          0.5
          ... 
200658    17.2
200659    17.3
200660    17.4
200661    17.5
200662    17.6
Name: ball, Length: 173644, dtype: float64

# Feature Engineering

In [14]:
df['overs']=df['ball']

In [15]:
df['ball'] = df['ball'].astype(str)

In [16]:
df['over']=df['ball'].str[0:-1]

In [17]:
df['over']=df['over'].str.replace('.', '')

In [18]:
df['ball']=df['ball'].str[-1:]

In [19]:
df=df[['match_id','innings','batting_team','bowling_team','overs','over','ball','Total_runs','player_dismissed','over_type','number_of_dotballs']]

In [20]:
df.head(5)

,match_id,innings,batting_team,bowling_team,overs,over,ball,Total_runs,player_dismissed,over_type,number_of_dotballs
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.1,0,1,1,0,PowerPlay,0.0
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.2,0,2,0,0,PowerPlay,1.0
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.3,0,3,1,0,PowerPlay,0.0
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.4,0,4,0,0,PowerPlay,1.0
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.5,0,5,0,0,PowerPlay,1.0


In [21]:
df.groupby(['match_id','innings'])['Total_runs'].transform('sum')

0         222
1         222
2         222
3         222
4         222
         ... 
200658    167
200659    167
200660    167
200661    167
200662    167
Name: Total_runs, Length: 173644, dtype: int64

In [22]:
df['total']=df.groupby(['match_id','innings'])['Total_runs'].transform('sum')

In [23]:
df['total_score']=df.groupby(['match_id', 'innings'])['Total_runs'].apply(lambda x: x.cumsum())

In [24]:
df.head(10)

,match_id,innings,batting_team,bowling_team,overs,over,ball,Total_runs,player_dismissed,over_type,number_of_dotballs,total,total_score
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.1,0,1,1,0,PowerPlay,0.0,222,1
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.2,0,2,0,0,PowerPlay,1.0,222,1
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.3,0,3,1,0,PowerPlay,0.0,222,2
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.4,0,4,0,0,PowerPlay,1.0,222,2
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.5,0,5,0,0,PowerPlay,1.0,222,2
5,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.6,0,6,0,0,PowerPlay,1.0,222,2
6,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.7,0,7,1,0,PowerPlay,0.0,222,3
7,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,1.1,1,1,0,0,PowerPlay,1.0,222,3
8,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,1.2,1,2,4,0,PowerPlay,0.0,222,7
9,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,1.3,1,3,4,0,PowerPlay,0.0,222,11


# Runs scored in previous 30 balls

In [25]:
#tmp=df.groupby(['match_id','innings','over_type'])['Total_runs'].transform(lambda x: x.cumsum()).reset_index()

In [26]:
tmp=df.groupby(['match_id','innings'])['Total_runs'].rolling(min_periods=1, window=30).sum().reset_index()

In [27]:
df['prev_30_runs']=tmp['Total_runs'].to_list()

# Wickets fallen in previous 30 balls

In [28]:
df['player_dismissed'].unique()

array(['0', 'SC Ganguly', 'RT Ponting', 'DJ Hussey', 'R Dravid',
       'V Kohli', 'JH Kallis', 'W Jaffer', 'MV Boucher', 'B Akhil',
       'CL White', 'AA Noffke', 'Z Khan', 'SB Joshi', 'PA Patel',
       'ML Hayden', 'MS Dhoni', 'SK Raina', 'JDP Oram', 'K Goel',
       'JR Hopes', 'Yuvraj Singh', 'KC Sangakkara', 'T Kohli',
       'YK Pathan', 'SR Watson', 'DS Lehmann', 'M Kaif', 'M Rawat',
       'RA Jadeja', 'SK Warne', 'V Sehwag', 'L Ronchi', 'DJ Thornely',
       'ST Jayasuriya', 'PR Shah', 'RV Uthappa', 'AM Nayar', 'SM Pollock',
       'S Chanderpaul', 'LRPL Taylor', 'Y Venugopal Rao', 'VVS Laxman',
       'AC Gilchrist', 'RG Sharma', 'SB Styris', 'AS Yadav', 'A Symonds',
       'WPUJC Vaas', 'SB Bangar', 'PP Ojha', 'BB McCullum', 'WP Saha',
       'Mohammad Hafeez', 'DPMD Jayawardene', 'IK Pathan', 'B Lee',
       'S Sohal', 'Kamran Akmal', 'Shahid Afridi', 'G Gambhir',
       'MEK Hussey', 'DJ Bravo', 'MA Khote', 'Harbhajan Singh',
       'GC Smith', 'D Salunkhe', 'RR Sarwan',

In [29]:
df['player_dismissed']=np.where(df['player_dismissed']==0, 0, 1)

In [30]:
df['total_wickets']=df.groupby(['match_id', 'innings'])['player_dismissed'].apply(lambda x: x.cumsum())

In [31]:
tmp=df.groupby(['match_id','innings'])['player_dismissed'].rolling(min_periods=1, window=30).sum().reset_index()
#tmp=tmp=df.groupby(['match_id','innings','over_type'])['player_dismissed'].transform(lambda x: x.cumsum()).reset_index()
df['prev_30_wickets']=tmp['player_dismissed'].to_list()

# Dot balls in previous 30 balls

In [32]:
tmp=df.groupby(['match_id','innings'])['number_of_dotballs'].rolling(min_periods=1, window=30).sum().reset_index()
#tmp=df.groupby(['match_id','innings','over_type'])['number_of_dotballs'].transform(lambda x: x.cumsum()).reset_index()

df['prev_30_dot_balls']=tmp['number_of_dotballs'].to_list()

# Boundaries in previous 30 balls

In [33]:
df['prev_30_boundaries']=df['Total_runs']
df['prev_30_boundaries']=np.where(df['prev_30_boundaries']>3, 1, 0)

tmp=df.groupby(['match_id','innings'])['prev_30_boundaries'].rolling(min_periods=1, window=30).sum().reset_index()
#tmp=df.groupby(['match_id','innings','over_type'])['prev_30_boundaries'].transform(lambda x: x.cumsum()).reset_index()

df['prev_30_boundaries']=tmp['prev_30_boundaries'].to_list()

In [34]:
df.dtypes

match_id                int64
innings                 int64
batting_team           object
bowling_team           object
overs                 float64
over                   object
ball                   object
Total_runs              int64
player_dismissed        int64
over_type              object
number_of_dotballs    float64
total                   int64
total_score             int64
prev_30_runs          float64
total_wickets           int64
prev_30_wickets       float64
prev_30_dot_balls     float64
prev_30_boundaries    float64
dtype: object

In [35]:
convert_dict = {'prev_30_runs': int, 
                'prev_30_wickets': int,
                'prev_30_dot_balls':int,
                'prev_30_boundaries':int
               } 
  
df = df.astype(convert_dict)

In [36]:
df=pd.get_dummies(data=df, columns=['batting_team', 'bowling_team'])

In [37]:
df.columns

Index(['match_id', 'innings', 'overs', 'over', 'ball', 'Total_runs',
       'player_dismissed', 'over_type', 'number_of_dotballs', 'total',
       'total_score', 'prev_30_runs', 'total_wickets', 'prev_30_wickets',
       'prev_30_dot_balls', 'prev_30_boundaries',
       'batting_team_Chennai Super Kings', 'batting_team_Delhi Capitals',
       'batting_team_Kolkata Knight Riders', 'batting_team_Mumbai Indians',
       'batting_team_Punjab Kings', 'batting_team_Rajasthan Royals',
       'batting_team_Royal Challengers Bangalore',
       'batting_team_Sunrisers Hyderabad', 'bowling_team_Chennai Super Kings',
       'bowling_team_Delhi Capitals', 'bowling_team_Kolkata Knight Riders',
       'bowling_team_Mumbai Indians', 'bowling_team_Punjab Kings',
       'bowling_team_Rajasthan Royals',
       'bowling_team_Royal Challengers Bangalore',
       'bowling_team_Sunrisers Hyderabad'],
      dtype='object')

In [38]:
df=df[['match_id','batting_team_Chennai Super Kings', 'batting_team_Delhi Capitals',
       'batting_team_Kolkata Knight Riders', 'batting_team_Mumbai Indians',
       'batting_team_Punjab Kings', 'batting_team_Rajasthan Royals',
       'batting_team_Royal Challengers Bangalore',
       'batting_team_Sunrisers Hyderabad', 'bowling_team_Chennai Super Kings',
       'bowling_team_Delhi Capitals', 'bowling_team_Kolkata Knight Riders',
       'bowling_team_Mumbai Indians', 'bowling_team_Punjab Kings',
       'bowling_team_Rajasthan Royals',
       'bowling_team_Royal Challengers Bangalore',
       'bowling_team_Sunrisers Hyderabad','overs','total_score','total_wickets','prev_30_runs','prev_30_wickets','prev_30_dot_balls','prev_30_boundaries','total']]

# train_test_split

In [39]:
X = df.drop(labels=['total','match_id'], axis=1)
y = df['total'].values

In [40]:
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=42,stratify=y)

In [41]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(130233, 23) (43411, 23) (130233,) (43411,)


# Training Model

In [42]:
LR_model = LinearRegression()
LR_model.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [43]:
# Creating a pickle file for the classifier
filename = 'lr-model.pkl'
pickle.dump(LR_model, open(filename, 'wb'))

In [44]:
prediction=LR_model.predict(X_test)
mean_absolute_error (y_test, prediction)

17.39413100818986

In [45]:
# Function to predict scores :

def score_prediction(Bat_Team,Bowl_Team,overs,total_score,total_wickets,prev_runs_30,prev_wickets_30,prev_30_dot_balls,prev_30_boundaries,model):

    temp_array = list()

    if Bat_Team == 'CSK':
        temp_array = temp_array + [1,0,0,0,0,0,0,0]
    elif Bat_Team == 'DC':
        temp_array = temp_array + [0,1,0,0,0,0,0,0]
    elif Bat_Team == 'KKR':
        temp_array = temp_array + [0,0,1,0,0,0,0,0]
    elif Bat_Team == 'MI':
        temp_array = temp_array + [0,0,0,1,0,0,0,0]
    elif Bat_Team == 'PK':
        temp_array = temp_array + [0,0,0,0,1,0,0,0]
    elif Bat_Team == 'RR':
        temp_array = temp_array + [0,0,0,0,0,1,0,0]
    elif Bat_Team == 'RCB':
        temp_array = temp_array + [0,0,0,0,0,0,1,0]
    elif Bat_Team == 'SRH':
        temp_array = temp_array + [0,0,0,0,0,0,0,1]
  
    if Bowl_Team == 'CSK':
        temp_array = temp_array + [1,0,0,0,0,0,0,0]
    elif Bowl_Team == 'DC':
        temp_array = temp_array + [0,1,0,0,0,0,0,0]
    elif Bowl_Team == 'KKR':
        temp_array = temp_array + [0,0,1,0,0,0,0,0]
    elif Bowl_Team == 'MI':
        temp_array = temp_array + [0,0,0,1,0,0,0,0]
    elif Bowl_Team == 'PK':
        temp_array = temp_array + [0,0,0,0,1,0,0,0]
    elif Bowl_Team == 'RR':
        temp_array = temp_array + [0,0,0,0,0,1,0,0]
    elif Bowl_Team == 'RCB':
        temp_array = temp_array + [0,0,0,0,0,0,1,0]
    elif Bowl_Team == 'SRH':
        temp_array = temp_array + [0,0,0,0,0,0,0,1]
    
    temp_array = temp_array + [overs,total_score,total_wickets,prev_runs_30,prev_wickets_30,prev_30_dot_balls,prev_30_boundaries]
    data = np.array([temp_array])
    my_prediction = int(model.predict(data))

    print('Predicted score: ',my_prediction)
    print('Predicted score range: ',my_prediction - 18, 'to' , my_prediction + 18)

In [46]:
Bat_Team = 'MI' 

Bowl_Team = 'CSK' 

overs = 10

total_score = 100 # current score

total_wickets = 1 # current wicket 

prev_runs_30 = 50 # runs in last 30 balls

prev_wickets_30 = 1 # wickets in last 30 balls

prev_30_dot_balls = 8 # dots in last 30 balls

prev_30_boundaries = 5 # boundaries in last 30 balls

model=LR_model
score_prediction(Bat_Team,Bowl_Team,overs,total_score,total_wickets,
                         prev_runs_30,prev_wickets_30,prev_30_dot_balls,prev_30_boundaries,model)

Predicted score:  220
Predicted score range:  202 to 238


In [47]:
xg_reg = xgb.XGBRegressor()
xg_reg.fit(X_train,y_train)
prediction = xg_reg.predict(X_test)
mean_absolute_error (y_test, prediction)
filename = 'XGB-model.pkl'
pickle.dump(LR_model, open(filename, 'wb'))

[12:01:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [50]:
names = xg_reg.get_booster().feature_names

In [58]:
def xgboostscore_prediction(Bat_Team,Bowl_Team,overs,total_score,total_wickets,prev_runs_30,prev_wickets_30,prev_30_dot_balls,prev_30_boundaries):

    temp_array = list()

    if Bat_Team == 'CSK':
        temp_array = temp_array + [1,0,0,0,0,0,0,0]
    elif Bat_Team == 'DC':
        temp_array = temp_array + [0,1,0,0,0,0,0,0]
    elif Bat_Team == 'KKR':
        temp_array = temp_array + [0,0,1,0,0,0,0,0]
    elif Bat_Team == 'MI':
        temp_array = temp_array + [0,0,0,1,0,0,0,0]
    elif Bat_Team == 'PK':
        temp_array = temp_array + [0,0,0,0,1,0,0,0]
    elif Bat_Team == 'RR':
        temp_array = temp_array + [0,0,0,0,0,1,0,0]
    elif Bat_Team == 'RCB':
        temp_array = temp_array + [0,0,0,0,0,0,1,0]
    elif Bat_Team == 'SRH':
        temp_array = temp_array + [0,0,0,0,0,0,0,1]
  
    if Bowl_Team == 'CSK':
        temp_array = temp_array + [1,0,0,0,0,0,0,0]
    elif Bowl_Team == 'DC':
        temp_array = temp_array + [0,1,0,0,0,0,0,0]
    elif Bowl_Team == 'KKR':
        temp_array = temp_array + [0,0,1,0,0,0,0,0]
    elif Bowl_Team == 'MI':
        temp_array = temp_array + [0,0,0,1,0,0,0,0]
    elif Bowl_Team == 'PK':
        temp_array = temp_array + [0,0,0,0,1,0,0,0]
    elif Bowl_Team == 'RR':
        temp_array = temp_array + [0,0,0,0,0,1,0,0]
    elif Bowl_Team == 'RCB':
        temp_array = temp_array + [0,0,0,0,0,0,1,0]
    elif Bowl_Team == 'SRH':
        temp_array = temp_array + [0,0,0,0,0,0,0,1]
    




    temp_array = temp_array + [overs,total_score,total_wickets,prev_runs_30,prev_wickets_30,prev_30_dot_balls,prev_30_boundaries]
    
    data = np.array([temp_array])
    data=pd.DataFrame(data,columns=names)
    my_prediction = int(xg_reg.predict(data))

    print('Predicted score: ',my_prediction)
    print('Predicted score range: ',my_prediction - 18, 'to' , my_prediction + 18)

In [60]:
Bat_Team = 'CSK' 

Bowl_Team = 'MI' 

overs = 10

total_score = 50 # current score

total_wickets = 1 # current wicket 

prev_runs_30 = 20 # runs in last 30 balls

prev_wickets_30 = 1 # wickets in last 30 balls

prev_30_dot_balls = 8 # dots in last 30 balls

prev_30_boundaries = 5 # boundaries in last 30 balls

#model=xg_reg
xgboostscore_prediction(Bat_Team,Bowl_Team,overs,total_score,total_wickets,prev_runs_30,prev_wickets_30,prev_30_dot_balls,prev_30_boundaries)

Predicted score:  162
Predicted score range:  144 to 180
